<a href="https://colab.research.google.com/github/catalin-giuglan/BoneFractureDetection/blob/main/ProiectCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import os

model = YOLO('/content/drive/MyDrive/Proiect_CDv2/Proiect/runs/YOLOv8n_SimCLR_Fractures_6_FINAL-BEST/weights/best.pt')

base_results_dir = '/content/drive/MyDrive/Proiect_CDv2/Proiect/validation_results'

result = model.val(data='/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml', split='train', save_dir=os.path.join(base_results_dir, 'train_results'))

print(" REZULTATE FINALE - TRAIN SET ")
print(f"mAP50: {result.box.map50:.4f}")
print(f"mAP50-95: {result.box.map:.4f}")
print(f"Precision: {result.box.mp:.4f}")
print(f"Recall: {result.box.mr:.4f}")

result = model.val(data='/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml', split='test', save_dir=os.path.join(base_results_dir, 'test_results'))

print("\n REZULTATE FINALE - TEST SET ")
print(f"mAP50: {result.box.map50:.4f}")
print(f"mAP50-95: {result.box.map:.4f}")
print(f"Precision: {result.box.mp:.4f}")
print(f"Recall: {result.box.mr:.4f}")

result = model.val(data='/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml', split='val', save_dir=os.path.join(base_results_dir, 'val_results'))
print("\n REZULTATE FINALE - VALID SET ")
print(f"mAP50: {result.box.map50:.4f}")
print(f"mAP50-95: {result.box.map:.4f}")
print(f"Precision: {result.box.mp:.4f}")
print(f"Recall: {result.box.mr:.4f}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.7±0.1 ms, read: 0.0±0.0 MB/s, size: 11.6 KB)
val: Scanning /content/drive/.shortcut-targets-by-id/1bpEezeI4zyb7RHLcjibKYGZxgPHiQ-_Z/Proiect_CDv2/Proiect/train/labels... 754 images, 73 backgrounds, 0 corrupt: 38% ━━━━╸─────── 754/1984 2.8it/s 7:38<7:17

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import glob
from ultralytics import YOLO

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
DATA_PATH = "/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml"
TRAIN_IMAGES_DIR = "/content/drive/MyDrive/Proiect_CDv2/Proiect/train/images"
PROJECT_DIR = "/content/drive/MyDrive/Proiect_CDv2/Proiect/runs"
RUN_NAME = "YOLOv8n_SimCLR_Fractures_6"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class XRayContrastiveDataset(Dataset):
    def __init__(self, img_dir, transform):
        self.img_paths = glob.glob(os.path.join(img_dir, "*.jpg")) + glob.glob(os.path.join(img_dir, "*.png"))
        self.transform = transform
    def __len__(self): return len(self.img_paths)
    def __getitem__(self, idx):
        img = Image.open(self.img_paths[idx]).convert("RGB")
        return self.transform(img), self.transform(img)

class SimCLR_YOLO_Backbone(nn.Module):
    def __init__(self):
        super().__init__()
        base = YOLO("yolov8n.pt").model
        self.backbone = nn.Sequential(*list(base.children())[0][:10])
        self.projection_head = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(),
            nn.Linear(256, 128), nn.ReLU(), nn.Linear(128, 64)
        )
    def forward(self, x):
        return self.projection_head(self.backbone(x))

def nt_xent_loss(z1, z2, temp=0.1):
    z1, z2 = nn.functional.normalize(z1), nn.functional.normalize(z2)
    logits = torch.matmul(z1, z2.T) / temp
    labels = torch.arange(z1.size(0)).to(z1.device)
    return nn.functional.cross_entropy(logits, labels)

print("--- START SIMCLR PRE-TRAINING (Self-Supervised) ---")
aug = transforms.Compose([
    transforms.RandomResizedCrop(512),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor()
])

dataset = XRayContrastiveDataset(TRAIN_IMAGES_DIR, aug)
loader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)
simclr_model = SimCLR_YOLO_Backbone().to(DEVICE)
optimizer = torch.optim.Adam(simclr_model.parameters(), lr=1e-3)

simclr_model.train()
for epoch in range(60):
    total_loss = 0
    for (img1, img2) in loader:
        optimizer.zero_grad()
        z1, z2 = simclr_model(img1.to(DEVICE)), simclr_model(img2.to(DEVICE))
        loss = nt_xent_loss(z1, z2)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}")

torch.save(simclr_model.backbone.state_dict(), "temp_backbone.pth")
print("SimCLR Pre-training done. Weights saved.")

print("\n--- START YOLOv8 TRAINING (Supervised) ---")
model = YOLO("yolov8n.pt")

backbone_module = model.model.model[:10]

print("Loading SimCLR weights into backbone...")
backbone_module.load_state_dict(torch.load("temp_backbone.pth"), strict=False)

results = model.train(
    data=DATA_PATH,
    project=PROJECT_DIR,
    name=RUN_NAME,
    imgsz=256,
    batch=512,
    epochs=150,
    patience=30,
    optimizer="auto",
    mosaic=0.0,
    mixup=0.05,
    label_smoothing=0.05
)

val_results = model.val()
print(f"mAP50: {val_results.box.map50:.4f}")
print(f"mAP50-95: {val_results.box.map:.4f}")
print(f"Precision: {val_results.box.mp:.4f}")
print(f"Recall: {val_results.box.mr:.4f}")
model.export(format="onnx")

--- START SIMCLR PRE-TRAINING (Self-Supervised) ---
Epoch 1, Loss: 1.9884
Epoch 2, Loss: 1.6365
Epoch 3, Loss: 1.5307
Epoch 4, Loss: 1.4393
Epoch 5, Loss: 1.3706
Epoch 6, Loss: 1.4148
Epoch 7, Loss: 1.3868
Epoch 8, Loss: 1.3940
Epoch 9, Loss: 1.3028
Epoch 10, Loss: 1.3045
Epoch 11, Loss: 1.2729
Epoch 12, Loss: 1.2364
Epoch 13, Loss: 1.3253
Epoch 14, Loss: 1.2616
Epoch 15, Loss: 1.2497
Epoch 16, Loss: 1.2434
Epoch 17, Loss: 1.1597
Epoch 18, Loss: 1.2398
Epoch 19, Loss: 1.1960
Epoch 20, Loss: 1.2413
Epoch 21, Loss: 1.2091
Epoch 22, Loss: 1.2500
Epoch 23, Loss: 1.2184
Epoch 24, Loss: 1.2063
Epoch 25, Loss: 1.1872
Epoch 26, Loss: 1.2019
Epoch 27, Loss: 1.1674
Epoch 28, Loss: 1.1785
Epoch 29, Loss: 1.1837
Epoch 30, Loss: 1.1374
Epoch 31, Loss: 1.2601
Epoch 32, Loss: 1.1639
Epoch 33, Loss: 1.1827
Epoch 34, Loss: 1.1441
Epoch 35, Loss: 1.1613
Epoch 36, Loss: 1.1561
Epoch 37, Loss: 1.1589
Epoch 38, Loss: 1.1130
Epoch 39, Loss: 1.1404
Epoch 40, Loss: 1.1295
Epoch 41, Loss: 1.1559
Epoch 42, Loss

/usr/local/lib/python3.12/dist-packages/torch/onnx/_internal/torchscript_exporter/utils.py:1447: OnnxExporterWarning: Exporting to ONNX opset version 22 is not supported. by 'torch.onnx.export()'. The highest opset version supported is 20. To use a newer opset version, consider 'torch.onnx.export(..., dynamo=True)'. 
  warnings.warn(


ONNX: slimming with onnxslim 0.1.80...
ONNX: export success ✅ 9.8s, saved as '/content/drive/.shortcut-targets-by-id/1bpEezeI4zyb7RHLcjibKYGZxgPHiQ-_Z/Proiect_CDv2/Proiect/runs/YOLOv8n_SimCLR_Fractures_6/weights/best.onnx' (11.6 MB)

Export complete (10.0s)
Results saved to /content/drive/.shortcut-targets-by-id/1bpEezeI4zyb7RHLcjibKYGZxgPHiQ-_Z/Proiect_CDv2/Proiect/runs/YOLOv8n_SimCLR_Fractures_6/weights
Predict:         yolo predict task=detect model=/content/drive/.shortcut-targets-by-id/1bpEezeI4zyb7RHLcjibKYGZxgPHiQ-_Z/Proiect_CDv2/Proiect/runs/YOLOv8n_SimCLR_Fractures_6/weights/best.onnx imgsz=256  
Validate:        yolo val task=detect model=/content/drive/.shortcut-targets-by-id/1bpEezeI4zyb7RHLcjibKYGZxgPHiQ-_Z/Proiect_CDv2/Proiect/runs/YOLOv8n_SimCLR_Fractures_6/weights/best.onnx imgsz=256 data=/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml  
Visualize:       https://netron.app


'/content/drive/.shortcut-targets-by-id/1bpEezeI4zyb7RHLcjibKYGZxgPHiQ-_Z/Proiect_CDv2/Proiect/runs/YOLOv8n_SimCLR_Fractures_6/weights/best.onnx'

In [ ]:
from ultralytics import YOLO
import os

model = YOLO('/content/drive/MyDrive/Proiect_CDv2/Proiect/runs/YOLOv8n_SimCLR_Fractures_6/weights/best.pt')

result = model.val(data='/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml', split='train')

print(" REZULTATE FINALE - TRAIN SET ")
print(f"mAP50: {result.box.map50:.4f}")
print(f"mAP50-95: {result.box.map:.4f}")
print(f"Precision: {result.box.mp:.4f}")
print(f"Recall: {result.box.mr:.4f}")

result = model.val(data='/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml', split='test')

print("\n REZULTATE FINALE - TEST SET ")
print(f"mAP50: {result.box.map50:.4f}")
print(f"mAP50-95: {result.box.map:.4f}")
print(f"Precision: {result.box.mp:.4f}")
print(f"Recall: {result.box.mr:.4f}")

result = model.val(data='/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml', split='val')
print("\n REZULTATE FINALE - VALID SET ")
print(f"mAP50: {result.box.map50:.4f}")
print(f"mAP50-95: {result.box.map:.4f}")
print(f"Precision: {result.box.mp:.4f}")
print(f"Recall: {result.box.mr:.4f}")

Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.6±0.1 ms, read: 1.7±0.8 MB/s, size: 8.5 KB)
val: Scanning /content/drive/.shortcut-targets-by-id/1bpEezeI4zyb7RHLcjibKYGZxgPHiQ-_Z/Proiect_CDv2/Proiect/train/labels... 1984 images, 180 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1984/1984 29.9it/s 1:06
val: New cache created: /content/drive/.shortcut-targets-by-id/1bpEezeI4zyb7RHLcjibKYGZxgPHiQ-_Z/Proiect_CDv2/Proiect/train/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 124/124 7.3it/s 17.0s
                   all       1984       2088      0.934       0.83       0.92      0.589
Speed: 0.3ms preprocess, 1.6ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /content/runs/detect/val2
 REZULTATE FINALE - TRAIN SET 
mAP50: 0.9196
mAP50-95: 0.5891
Precision

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import glob
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/Proiect_CDv2/Proiect/runs/YOLOv8n_SimCLR_Fractures_512batch&256size_1/weights/best.pt")

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
DATA_PATH = "/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml"
TRAIN_IMAGES_DIR = "/content/drive/MyDrive/Proiect_CDv2/Proiect/train/images"
PROJECT_DIR = "/content/drive/MyDrive/Proiect_CDv2/Proiect/runs"
RUN_NAME = "YOLOv8n_SimCLR_Fractures_512batch&256size_12"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

results = model.train(
    data=DATA_PATH,
    project=PROJECT_DIR,
    name=RUN_NAME,
    imgsz=256,
    batch=512,
    epochs=150,
    patience=30,
    optimizer="auto",
    mosaic=0.0,
    mixup=0.05,
    label_smoothing=0.05
)

val_results = model.val()
print(f"mAP50: {val_results.box.map50:.4f}")
print(f"mAP50-95: {val_results.box.map:.4f}")
print(f"Precision: {val_results.box.mp:.4f}")
print(f"Recall: {val_results.box.mr:.4f}")
model.export(format="onnx")